In [1]:
%%time

from pandas import DataFrame, Grouper
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import sys
from collections import defaultdict
from IPython.display import HTML

CPU times: user 381 ms, sys: 936 ms, total: 1.32 s
Wall time: 262 ms


In [2]:
%%time
# import our own stuff
sys.path.append("../python/src")

from tools import read_water_data, extract_features
from tools import get_oecd_iso3_code_mapping

from waterData import default_features

from worldbankApi import get_regionnames, fetch_series, default_series, get_all_countries

CPU times: user 74.8 ms, sys: 3.61 ms, total: 78.4 ms
Wall time: 78 ms


In [3]:
%%time
# set some options

datadir = "../data/"
cachedir = "../data/cache" 

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.29 µs


In [4]:
taz_correspondents_countries = tcc = {
    'Jordan': 'JOR',
    'Egypt, Arab Rep.': 'EGY',
    'Uganda': 'UGA',
    'Rwanda': 'RWA',
    'Kenya': 'KEN',
    'Sudan': 'SDN',
    'India': 'IND',
    'Bangladesh': 'BGD',
    'Colombia': 'COL',
    'Bolivia': 'BOL'
}

inverse_taz_correspondents_countries = itcc = {v: k for k, v in taz_correspondents_countries.items()}


In [5]:
%%time

display("warming up cache")
null = read_water_data(setname="fullset",datadir=datadir,cachedir=cachedir)
del null

'warming up cache'

Reading Datafrom cached file: ../data/cache/fullset.p
CPU times: user 4.83 s, sys: 1.75 s, total: 6.57 s
Wall time: 6.59 s


In [6]:
%%time

# read some data and get some mapping that may needs calculations

# the codemapping is generate over the full dataset to get all oecd-country-codes
# and from a fetch of the countries known to worldbank
codemapping = get_oecd_iso3_code_mapping(cachedir=cachedir,datadir=datadir)

# the data from oecd must be downloaded and saved as zips in the data-dir, check readme for links
oecddf = read_water_data(setname="fullset",datadir=datadir,cachedir=cachedir)


display("all data")
# tell me what it is
oecddf.describe()

Reading countrycode-mapping cached file: ../data/cache/oecdiso3.p
Reading Datafrom cached file: ../data/cache/fullset.p


'all data'

CPU times: user 10.5 s, sys: 3.04 s, total: 13.5 s
Wall time: 13.6 s


,USD_Commitment,USD_Disbursement,USD_Received,USD_Commitment_Defl,USD_Disbursement_Defl,USD_Received_Defl,USD_Adjustment,USD_Adjustment_Defl,USD_AmountUntied,USD_AmountPartialTied,...,TypeRepayment,NumberRepayment,Interest2,USD_Interest,USD_Outstanding,USD_Arrears_Principal,USD_Arrears_Interest,CapitalExpend,PSIflag,PSIAddType
count,3.051025e+06,3.506449e+06,1.980515e+06,3.051025e+06,3.506449e+06,1.980515e+06,54325.000000,54325.000000,1.822440e+06,963806.000000,...,542849.000000,553752.000000,328270.000000,1.232437e+06,867437.000000,620176.000000,628093.000000,32296.000000,2161.000000,711.000000
mean,1.732823e+00,9.959690e-01,4.593473e-01,2.103196e+00,1.056015e+00,4.927653e-01,-0.810849,-0.912068,1.016804e+00,0.339658,...,1.477022,1.869440,396.913882,2.375613e-01,8.368500,0.237621,0.283841,0.000090,1.987506,2.015471
std,2.047103e+01,1.487377e+01,7.843651e+00,2.414103e+01,1.565664e+01,8.306830e+00,12.451076,12.358834,1.542315e+01,6.633610,...,1.184970,0.551107,584.336396,3.609668e+00,74.482526,9.964379,5.574769,0.001934,0.111103,0.984979
min,-1.421530e+03,-2.258730e+02,-5.030000e-01,-1.300270e+03,-2.461000e+02,-6.852520e-01,-1995.000000,-1870.620000,-1.421530e+03,-1.787180,...,0.000000,0.000000,0.000000,-3.491230e+01,-0.507119,-3.033060,-0.149653,0.000000,1.000000,1.000000
25%,0.000000e+00,3.781470e-03,-0.000000e+00,0.000000e+00,3.927740e-03,-0.000000e+00,-0.089212,-0.093630,0.000000e+00,0.000000,...,1.000000,2.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000
50%,2.214890e-02,4.171300e-02,0.000000e+00,2.389890e-02,4.416200e-02,0.000000e+00,-0.010290,-0.010783,4.355780e-03,0.000000,...,1.000000,2.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,2.000000,2.000000
75%,2.008640e-01,2.386590e-01,-0.000000e+00,2.280270e-01,2.531470e-01,0.000000e+00,-0.000900,-0.000928,1.263540e-01,0.000000,...,1.000000,2.000000,750.000000,4.892500e-02,1.292290,0.000000,0.000000,0.000000,2.000000,3.000000
max,6.585080e+03,7.188450e+03,3.690330e+03,7.209450e+03,7.209450e+03,3.637680e+03,0.004700,0.004720,4.138840e+03,2053.390000,...,5.000000,52.000000,20000.000000,1.460130e+03,24053.000000,4317.120000,1586.170000,0.100000,2.000000,3.000000


In [7]:
%%time

# filter out useless / unrelevant data
df = oecddf[oecddf['USD_Commitment'] != 0.0]

df = df[df['USD_Commitment'].notnull()]

# tell me what is left
display("filtered for entries with commitment")
#display(df.describe())

df = df[df['FlowCode'].isin(['13','11'])]

display("only FolowCode 11 and 13")

#display(df.describe())

df = df[df['RecipientCode'].isin(codemapping)]
display("filter regions - only countries that can have an IncomeGroup-Classification")

#display(df.describe())


df = df[df['CommitmentDate'] > datetime(year=1995,month=12,day=31)]
df = df[df['CommitmentDate'] < datetime(year=2019,month=1,day=1)]

display("oecd historyical incomegroupdata is only available for the the years 1996-2019. filtering everything else")

display(df.describe())

# reduce to relevant features
df = extract_features(df,features=default_features)

# free the memory
del oecddf


'filtered for entries with commitment'

'only FolowCode 11 and 13'

'filter regions - only countries that can have an IncomeGroup-Classification'

'oecd historyical incomegroupdata is only available for the the years 1996-2019. filtering everything else'

,USD_Commitment,USD_Disbursement,USD_Received,USD_Commitment_Defl,USD_Disbursement_Defl,USD_Received_Defl,USD_Adjustment,USD_Adjustment_Defl,USD_AmountUntied,USD_AmountPartialTied,...,TypeRepayment,NumberRepayment,Interest2,USD_Interest,USD_Outstanding,USD_Arrears_Principal,USD_Arrears_Interest,CapitalExpend,PSIflag,PSIAddType
count,1.191322e+06,985673.000000,415491.000000,1.191322e+06,985673.000000,415491.000000,2.000000,2.000000,670509.000000,247973.000000,...,26190.000000,26569.000000,34839.000000,206380.000000,199329.000000,167047.000000,171905.000000,3228.000000,174.000000,2.0
mean,1.601822e+00,0.665421,0.008662,1.733270e+00,0.702612,0.009078,-2.767986,-2.767856,1.436260,0.276965,...,1.563154,1.827430,248.043074,0.015690,0.664081,0.003587,0.153497,0.000220,1.988506,3.0
std,1.640281e+01,11.583172,0.590591,1.722795e+01,12.191604,0.602133,3.908609,3.908793,16.429401,4.036799,...,1.405181,0.869447,534.832681,0.787091,56.052469,0.146133,1.704479,0.001575,0.106901,0.0
min,-2.005710e+01,-14.427400,0.000000,-2.180880e+01,-15.655900,0.000000,-5.531790,-5.531790,-20.057100,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,3.0
25%,1.343550e-02,0.007851,0.000000,1.448373e-02,0.008154,0.000000,-4.149888,-4.149823,0.003327,0.000000,...,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,3.0
50%,6.402245e-02,0.038414,0.000000,7.081125e-02,0.040749,0.000000,-2.767986,-2.767856,0.046104,0.000000,...,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,3.0
75%,3.292002e-01,0.169090,-0.000000,3.689865e-01,0.181482,-0.000000,-1.386084,-1.385888,0.289428,0.000000,...,1.000000,2.000000,470.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,3.0
max,4.087440e+03,4087.440000,155.161000,4.704100e+03,4704.100000,155.161000,-0.004182,-0.003921,4087.360000,875.123000,...,5.000000,12.000000,6300.000000,284.616000,24053.000000,14.391200,38.043100,0.024000,2.000000,3.0


CPU times: user 8.17 s, sys: 1.44 s, total: 9.61 s
Wall time: 9.66 s


In [8]:
mdg_series = {
    'ER.H2O.FWTL.ZS': 'Annual freshwater withdrawals, total (% of internal resources)',
    'ER.H2O.FWTL.K3': 'Annual freshwater withdrawals, total (billion cubic meters)',
    'ER.H2O.FWST.ZS': 'Level of water stress: freshwater withdrawal as a proportion of available freshwater resources',
    'SH.H2O.BASW.ZS': 'People using at least basic drinking water services (% of population)',
    'SG.H2O.TM30.HH.ZS': 'Households with water 30 minutes or longer away round trip (%)',
    'SG.H2O.TL30.HH.ZS': 'Households with water less than 30 minutes away round trip (%)',
    'SG.H2O.PRMS.HH.ZS': 'Households with water on the premises (%)',
    # 'SH.H2O.SAFE.ZS': 'Improved water source (% of population with access)', # not accessibale -> archiv
    'SH.H2O.SMDW.ZS': 'People using safely managed drinking water services (% of population)',
    # 'SH.STA.ACSN': 'Improved sanitation facilities (% of population with access)', #  not accessibale -> archiv
    'SP.POP.TOTL': 'Population, total',
#    'IN.POV.HH.DRNKNGWATER.TOTL.PCT': 'Households Access to Safe Drinking Water - Total (%) (india)', # Country Partnership Strategy for India (FY2013 - 17)
#    'IN.POV.HH.DRKNGWATER.WITHIN': 'Availability of drinking water source within the premises (india)',
#    'IN.POV.HH.DRKNGWATER.NEAR': 'Availability of drinking water source near the premises (india)',
#    'IN.POV.HH.DRKNGWATER.AWAY': 'Availability of drinking water from a source away (india)',
#    'IN.POV.HH.DRKNGWATER': 'Total households with drinking water facility (india)',
    'IE.PPN.WATR.CD': 'Public private partnerships investment in water and sanitation (current US$)',
    'IE.PPI.WATR.CD': 'Investment in water and sanitation with private participation (current US$)',
    'IC.WAT.GIFT.ZS': 'Expected to give gifts to get a water connection (% of firms)',
    'IC.FRM.CORR.CORR7': 'Percent of firms expected to give gifts to get a water connection',
    'IC.WAT.DURS': 'Delay in obtaining a water connections (days)',
    'IC.FRM.INFRA.IN6':'Number of water insufficiencies in a typical month',
#    'HOU.H2O.ACSN.ZS': 'Household Access to Safe Water (in % of total household) (indonesia)',
    'ER.H2O.INTR.PC': 'Renewable internal freshwater resources per capita (cubic meters)',
    'ER.H2O.INTR.K3': 'Renewable internal freshwater resources, total (billion cubic meters)',
#    '5.51.01.09.water': 'Access to water' # usless data
    'SI.POV.XPND.MD': 'Median daily per capita income or consumption expenditure (2011 PPP)',
    'SI.SPR.PCAP': 'Survey mean consumption or income per capita, total population (2011 PPP $ per day)',
    'AG.SRF.TOTL.K2': 'Surface area (sq. km)'
    
}

wdf = fetch_series(series=mdg_series,date="1999:2018")
countrynames = get_all_countries(cachedir=cachedir,includeaggs=True)
wdf.rename(columns=mdg_series,inplace=True)
#display(countrynames)


Reading Data from cached file: data/cache/ER.H2O.FWTL.ZS.p
Reading Data from cached file: data/cache/ER.H2O.FWTL.K3.p
Reading Data from cached file: data/cache/ER.H2O.FWST.ZS.p
Reading Data from cached file: data/cache/SH.H2O.BASW.ZS.p
Reading Data from cached file: data/cache/SG.H2O.TM30.HH.ZS.p
Reading Data from cached file: data/cache/SG.H2O.TL30.HH.ZS.p
Reading Data from cached file: data/cache/SG.H2O.PRMS.HH.ZS.p
Reading Data from cached file: data/cache/SH.H2O.SMDW.ZS.p
Reading Data from cached file: data/cache/SP.POP.TOTL.p
Reading Data from cached file: data/cache/IE.PPN.WATR.CD.p
Reading Data from cached file: data/cache/IE.PPI.WATR.CD.p
Reading Data from cached file: data/cache/IC.WAT.GIFT.ZS.p
Reading Data from cached file: data/cache/IC.FRM.CORR.CORR7.p
Reading Data from cached file: data/cache/IC.WAT.DURS.p
Reading Data from cached file: data/cache/IC.FRM.INFRA.IN6.p
Reading Data from cached file: data/cache/ER.H2O.INTR.PC.p
Reading Data from cached file: data/cache/ER.H2O

In [9]:
for i in itcc:
    d = defaultdict(dict)
    tmp = wdf[wdf['Country'] == i].copy()
    tmp = tmp.replace(0.0,np.nan)
    tmp = tmp.drop(columns=['Country'])
    tmp = tmp.set_index('Year')
    for j in tmp.columns:
        tmp2 = tmp[j]
        tmp2 = tmp2[tmp2.notna()].tail(1)
        
        #display(tmp2.reset_index().tail(1))
        #display(tmp2)
        tmp2 = tmp2.reset_index()
        #display(tmp2)
        try:
            d[j]['Year']=tmp2['Year'][0]
            d[j]['Value']=tmp2[j][0]
        except:
            d[j]['Year'] = np.nan
            d[j]['Value'] = np.nan
            pass
    htmlout = """<h2>%s<h2>
    <table>
    <tr><th>Stat</th><th>Year</th><th>Value</th></tr>
    """ %(itcc[i])
    for j in tmp.columns:
        htmlout = htmlout + """
        <tr><td> %s </td><td> %s </td><td> %.2f </td></tr>
        """ %(j.replace("$","&#36;").replace("%","&#37;"),d[j]['Year'],d[j]['Value'])
    
    ### some stats on grants
    tmp = df[df['RecipientCode'] == codemapping[i]].copy()
    tmp = tmp[tmp["CommitmentDate"] > datetime(year=1995,month=12,day=31)]
    tmp = tmp[tmp["CommitmentDate"] < datetime(year=2019,month=1,day=1)]
    j = 'sum of Grants mUSD'
    d[j]['Year'] = "1996-2018"
    d[j]['Value'] = float(tmp['USD_Commitment_Defl'].sum())
    htmlout = htmlout + """
    <tr><td> %s </td><td> %s </td><td> %.2f </td></tr>
    """ %(j,d[j]['Year'],d[j]['Value'])

    j = 'sum of Grants mUSD for water'
    d[j]['Year'] = "1996-2018"
    d[j]['Value'] = float(tmp[tmp['SectorCode']=='140']['USD_Commitment_Defl'].sum())
    htmlout = htmlout + """
    <tr><td> %s </td><td> %s </td><td> %.2f </td></tr>
    """ %(j,d[j]['Year'],d[j]['Value'])
        
    tmp = df[df['RecipientCode'] == codemapping[i]].copy()
    tmp = tmp[tmp['DonorCode'] == "5"]
    tmp = tmp[tmp["CommitmentDate"] > datetime(year=1995,month=12,day=31)]
    tmp = tmp[tmp["CommitmentDate"] < datetime(year=2019,month=1,day=1)]
    j = 'sum of Grants mUSD by germany'
    d[j]['Year'] = "1996-2018"
    d[j]['Value'] = float(tmp['USD_Commitment_Defl'].sum())
    htmlout = htmlout + """
    <tr><td> %s </td><td> %s </td><td> %.2f </td></tr>
    """ %(j,d[j]['Year'],d[j]['Value'])
    
    j = 'sum of Grants mUSD for water germany'
    d[j]['Year'] = "1996-2018"
    d[j]['Value'] = float(tmp[tmp['SectorCode']=='140']['USD_Commitment_Defl'].sum())
    htmlout = htmlout + """
    <tr><td> %s </td><td> %s </td><td> %.2f </td></tr>
    """ %(j,d[j]['Year'],d[j]['Value'])
    
    tmp = df[df['RecipientCode'] == codemapping[i]].copy()
    tmp = tmp[tmp["CommitmentDate"] > datetime(year=2009,month=12,day=31)]
    tmp = tmp[tmp["CommitmentDate"] < datetime(year=2019,month=1,day=1)]
    j = 'sum of Grants mUSD'
    d[j]['Year'] = "2010-2018"
    d[j]['Value'] = float(tmp['USD_Commitment_Defl'].sum())
    htmlout = htmlout + """
    <tr><td> %s </td><td> %s </td><td> %.2f </td></tr>
    """ %(j,d[j]['Year'],d[j]['Value'])

    j = 'sum of Grants mUSD for water'
    d[j]['Year'] = "2010-2018"
    d[j]['Value'] = float(tmp[tmp['SectorCode']=='140']['USD_Commitment_Defl'].sum())
    htmlout = htmlout + """
    <tr><td> %s </td><td> %s </td><td> %.2f </td></tr>
    """ %(j,d[j]['Year'],d[j]['Value'])

    tmp = df[df['RecipientCode'] == codemapping[i]].copy()
    tmp = tmp[tmp['DonorCode'] == "5"]
    tmp = tmp[tmp["CommitmentDate"] > datetime(year=1995,month=12,day=31)]
    tmp = tmp[tmp["CommitmentDate"] < datetime(year=2019,month=1,day=1)]
    j = 'sum of Grants mUSD by germany'
    d[j]['Year'] = "2010-2018"
    d[j]['Value'] = float(tmp['USD_Commitment_Defl'].sum())
    htmlout = htmlout + """
    <tr><td> %s </td><td> %s </td><td> %.2f </td></tr>
    """ %(j,d[j]['Year'],d[j]['Value'])

    j = 'sum of Grants mUSD for water by germany'
    d[j]['Year'] = "2010-2018"
    d[j]['Value'] = float(tmp[tmp['SectorCode']=='140']['USD_Commitment_Defl'].sum())
    htmlout = htmlout + """
    <tr><td> %s </td><td> %s </td><td> %.2f </td></tr>
    """ %(j,d[j]['Year'],d[j]['Value'])
        
    htmlout = htmlout + "</table>"
    display(HTML(htmlout))
    

Stat,Year,Value
"Annual freshwater withdrawals, total (% of internal resources)",2014,124.49
"Annual freshwater withdrawals, total (billion cubic meters)",2015,1.10
Level of water stress: freshwater withdrawal as a proportion of available freshwater resources,2015,150.90
People using at least basic drinking water services (% of population),2017,98.94
Households with water 30 minutes or longer away round trip (%),2018,0.10
Households with water less than 30 minutes away round trip (%),2018,0.40
Households with water on the premises (%),2018,99.30
People using safely managed drinking water services (% of population),2017,93.82
"Population, total",2019,10101694.00
Public private partnerships investment in water and sanitation (current US$),2012,223000000.00


Stat,Year,Value
"Annual freshwater withdrawals, total (% of internal resources)",2014,4100.00
"Annual freshwater withdrawals, total (billion cubic meters)",2010,73.80
Level of water stress: freshwater withdrawal as a proportion of available freshwater resources,2014,159.85
People using at least basic drinking water services (% of population),2017,99.11
Households with water 30 minutes or longer away round trip (%),2014,0.60
Households with water less than 30 minutes away round trip (%),2014,6.10
Households with water on the premises (%),2014,93.20
People using safely managed drinking water services (% of population),nan,nan
"Population, total",2019,100388073.00
Public private partnerships investment in water and sanitation (current US$),2010,475000000.00


Stat,Year,Value
"Annual freshwater withdrawals, total (% of internal resources)",2012,1.63
"Annual freshwater withdrawals, total (billion cubic meters)",2008,0.64
Level of water stress: freshwater withdrawal as a proportion of available freshwater resources,2014,1.33
People using at least basic drinking water services (% of population),2017,49.10
Households with water 30 minutes or longer away round trip (%),2019,29.50
Households with water less than 30 minutes away round trip (%),2019,49.30
Households with water on the premises (%),2019,20.20
People using safely managed drinking water services (% of population),2017,7.07
"Population, total",2019,44269594.00
Public private partnerships investment in water and sanitation (current US$),nan,nan


Stat,Year,Value
"Annual freshwater withdrawals, total (% of internal resources)",2002,1.58
"Annual freshwater withdrawals, total (billion cubic meters)",2000,0.15
Level of water stress: freshwater withdrawal as a proportion of available freshwater resources,2014,1.44
People using at least basic drinking water services (% of population),2017,57.71
Households with water 30 minutes or longer away round trip (%),2017,27.00
Households with water less than 30 minutes away round trip (%),2017,58.40
Households with water on the premises (%),2017,14.10
People using safely managed drinking water services (% of population),nan,nan
"Population, total",2019,12626950.00
Public private partnerships investment in water and sanitation (current US$),2017,60000000.00


Stat,Year,Value
"Annual freshwater withdrawals, total (% of internal resources)",2014,15.55
"Annual freshwater withdrawals, total (billion cubic meters)",2010,3.22
Level of water stress: freshwater withdrawal as a proportion of available freshwater resources,2014,14.31
People using at least basic drinking water services (% of population),2017,58.92
Households with water 30 minutes or longer away round trip (%),2015,12.10
Households with water less than 30 minutes away round trip (%),2015,39.50
Households with water on the premises (%),2015,47.50
People using safely managed drinking water services (% of population),nan,nan
"Population, total",2019,52573973.00
Public private partnerships investment in water and sanitation (current US$),nan,nan


Stat,Year,Value
"Annual freshwater withdrawals, total (% of internal resources)",2014,673.25
"Annual freshwater withdrawals, total (billion cubic meters)",2011,26.93
Level of water stress: freshwater withdrawal as a proportion of available freshwater resources,2014,93.67
People using at least basic drinking water services (% of population),2017,60.27
Households with water 30 minutes or longer away round trip (%),nan,nan
Households with water less than 30 minutes away round trip (%),nan,nan
Households with water on the premises (%),nan,nan
People using safely managed drinking water services (% of population),nan,nan
"Population, total",2019,42813238.00
Public private partnerships investment in water and sanitation (current US$),2007,120700000.00


Stat,Year,Value
"Annual freshwater withdrawals, total (% of internal resources)",2014,44.78
"Annual freshwater withdrawals, total (billion cubic meters)",2010,647.50
Level of water stress: freshwater withdrawal as a proportion of available freshwater resources,2014,44.53
People using at least basic drinking water services (% of population),2017,92.67
Households with water 30 minutes or longer away round trip (%),2016,2.30
Households with water less than 30 minutes away round trip (%),2016,31.40
Households with water on the premises (%),2016,66.30
People using safely managed drinking water services (% of population),nan,nan
"Population, total",2019,1366417754.00
Public private partnerships investment in water and sanitation (current US$),2018,583300000.00


Stat,Year,Value
"Annual freshwater withdrawals, total (% of internal resources)",2012,34.16
"Annual freshwater withdrawals, total (billion cubic meters)",2008,35.87
Level of water stress: freshwater withdrawal as a proportion of available freshwater resources,2014,3.79
People using at least basic drinking water services (% of population),2017,97.02
Households with water 30 minutes or longer away round trip (%),2014,1.60
Households with water less than 30 minutes away round trip (%),2014,22.70
Households with water on the premises (%),2014,75.50
People using safely managed drinking water services (% of population),2017,55.44
"Population, total",2019,163046161.00
Public private partnerships investment in water and sanitation (current US$),2019,72000000.00


Stat,Year,Value
"Annual freshwater withdrawals, total (% of internal resources)",2012,0.55
"Annual freshwater withdrawals, total (billion cubic meters)",2008,11.77
Level of water stress: freshwater withdrawal as a proportion of available freshwater resources,2014,0.86
People using at least basic drinking water services (% of population),2017,97.30
Households with water 30 minutes or longer away round trip (%),2015,0.60
Households with water less than 30 minutes away round trip (%),2015,3.30
Households with water on the premises (%),2015,95.90
People using safely managed drinking water services (% of population),2017,73.23
"Population, total",2019,50339443.00
Public private partnerships investment in water and sanitation (current US$),2013,129000000.00


Stat,Year,Value
"Annual freshwater withdrawals, total (% of internal resources)",2012,0.69
"Annual freshwater withdrawals, total (billion cubic meters)",2009,2.09
Level of water stress: freshwater withdrawal as a proportion of available freshwater resources,2014,0.53
People using at least basic drinking water services (% of population),2017,92.85
Households with water 30 minutes or longer away round trip (%),2008,0.70
Households with water less than 30 minutes away round trip (%),2008,11.20
Households with water on the premises (%),2008,88.00
People using safely managed drinking water services (% of population),nan,nan
"Population, total",2019,11513100.00
Public private partnerships investment in water and sanitation (current US$),1999,320000000.00
